In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.model_selection import train_test_split as skl_train_test_split
from surprise import KNNBasic,NormalPredictor,BaselineOnly,NMF
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
#Carga de archivos
reviews_data = pd.read_parquet('reviews.parquet')
users_data = pd.read_parquet('users.parquet')
business_data = pd.read_parquet('business.parquet')

In [3]:
#Información sobre las varaibles de cada archivo
business_info = business_data.info()
reviews_info = reviews_data.info()
users_info = users_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348 entries, 0 to 1347
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   1348 non-null   object 
 1   name          1348 non-null   object 
 2   address       1348 non-null   object 
 3   city          1348 non-null   object 
 4   state         1348 non-null   object 
 5   postal_code   1348 non-null   object 
 6   latitude      1348 non-null   float64
 7   longitude     1348 non-null   float64
 8   stars         1348 non-null   float64
 9   review_count  1348 non-null   int64  
 10  is_open       1348 non-null   int64  
 11  attributes    1348 non-null   object 
 12  categories    1348 non-null   object 
 13  hours         1337 non-null   object 
dtypes: float64(3), int64(2), object(9)
memory usage: 147.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23734 entries, 0 to 23733
Data columns (total 9 columns):
 #   Column       Non-Null Count 

In [4]:
reviews_data.describe()

,stars,useful,funny,cool
count,23734.000000,23734.000000,23734.000000,23734.000000
mean,3.972655,5.047485,2.377391,3.842504
std,0.921731,8.367556,5.297148,7.577344
min,1.000000,0.000000,0.000000,0.000000
25%,3.000000,1.000000,0.000000,0.000000
50%,4.000000,3.000000,1.000000,2.000000
75%,5.000000,6.000000,3.000000,4.000000
max,5.000000,222.000000,159.000000,207.000000


In [5]:
users_data.describe()

,review_count,useful,funny,cool,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
count,1949.000000,1949.000000,1949.000000,1949.000000,1949.000000,1949.000000,1949.000000,1949.000000,1949.000000,1949.000000,1949.000000,1949.000000,1949.000000,1949.000000,1949.000000,1949.000000,1949.000000
mean,1580.809133,8995.987173,4744.301180,7070.395587,238.477168,3.821503,553.441252,66.765008,59.649564,41.480246,26.009749,363.787583,969.196511,828.325808,828.325808,287.470498,442.244741
std,988.137006,15510.066645,10321.129862,14227.956109,429.622712,0.301630,1735.560019,361.164540,417.145419,335.805956,302.641462,1546.124090,3137.870609,2323.167604,2323.167604,816.799060,2654.501317
min,1001.000000,157.000000,15.000000,51.000000,1.000000,2.580000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1122.000000,2273.000000,781.000000,1205.000000,74.000000,3.630000,31.000000,8.000000,4.000000,1.000000,0.000000,39.000000,60.000000,65.000000,65.000000,36.000000,12.000000
50%,1309.000000,4403.000000,1835.000000,2850.000000,129.000000,3.820000,97.000000,17.000000,9.000000,5.000000,3.000000,96.000000,168.000000,199.000000,199.000000,90.000000,48.000000
75%,1701.000000,9471.000000,4646.000000,6998.000000,240.000000,4.020000,382.000000,46.000000,29.000000,20.000000,12.000000,301.000000,669.000000,686.000000,686.000000,243.000000,196.000000
max,17473.000000,206296.000000,185823.000000,199878.000000,12497.000000,4.990000,25784.000000,13501.000000,14180.000000,13654.000000,12669.000000,59031.000000,101097.000000,49967.000000,49967.000000,15934.000000,82630.000000


In [6]:
business_data.describe()

,latitude,longitude,stars,review_count,is_open
count,1348.000000,1348.000000,1348.000000,1348.000000,1348.000000
mean,34.901965,-90.548043,3.987760,924.215134,0.908012
std,4.828198,14.892444,0.453108,670.957904,0.289116
min,27.688697,-119.962691,1.000000,501.000000,0.000000
25%,29.954951,-90.214027,4.000000,577.750000,1.000000
50%,36.151327,-86.783396,4.000000,702.000000,1.000000
75%,39.907649,-82.428167,4.500000,981.000000,1.000000
max,43.636694,-74.716630,5.000000,7568.000000,1.000000


In [7]:
reviews_data.head(5)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Ltl-K9qNkYZfceRtu6Vr4A,nkN_do3fJ9xekchVC-v68A,8A5LSwsKK5vXDH1M3_Tf1w,5,6,1,6,Bacara is amazing and makes for an indulgent r...,2005-03-29 19:29:36
1,9qnI6MbB82Fy_67eYBAoMg,nkN_do3fJ9xekchVC-v68A,U3grYFIeu6RgAAQgdriHww,5,9,2,3,"Not the easiest place to find, but just what I...",2005-03-29 19:42:12
2,d4-VjTAvvmT1GVV40hR1rQ,nkN_do3fJ9xekchVC-v68A,UjOq8dBVNv9weHrVDqm3kQ,1,4,0,0,Times like these I wish I could give negative ...,2005-03-29 19:53:58
3,v3YVV8C6S46Yq5mHSLvt8g,nkN_do3fJ9xekchVC-v68A,CoZ2mpsMBP8HUG1ymKoZTg,3,2,0,1,Started with ceviche which was fresh and excel...,2005-03-31 01:45:05
4,h-ACmkD5yYpD_AW5kRs9zA,Bf87HcPERF9yiSjb2tQBqw,ld_H5-FpZOWm_tkzwkPYQQ,5,0,1,6,"Three pulls, a total of $1.25 wagered, payout ...",2005-05-15 09:58:17


In [8]:
users_data.head(5)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
5,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1221,2005-03-14 20:26:35,14953,9940,11211,"2006,2007,2008,2009,2010,2011,2012,2013,2014","xBDpTUbai0DXrvxCe3X16Q, 7GPNBO496aecrjJfW6UWtg...",1357,...,163,191,361,147,1212,5696,2543,2543,815,323
23,MGPQVLsODMm9ZtYQW-g_OA,Jelena,1807,2008-01-19 22:50:00,17331,10461,13734,"2008,2009,2010,2011,2012,2013,2014,2015,2016,2...","HpJzEFN1kyuH6VAF_usqKg, F_NG8ZaNVriV_hujoOejIA...",828,...,167,206,395,130,915,3396,1915,1915,696,470
32,NIhcRW6DWvk1JQhDhXwgOQ,Lia,2288,2005-12-30 13:47:19,12773,4199,7971,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...","T1upaPMzuW7pNj74fO1rjA, CP28puvAEimt4ziuGTDaHA...",345,...,63,46,47,21,518,773,867,867,310,231
38,QJI9OSEn6ujRCtrX06vs1w,J,1982,2007-03-19 18:15:32,14004,6466,9879,2010,"RyPeT_ICAtX8ah9dhDpEFw, W8r4aKPZFT3GPIQQDbqB6Q...",316,...,98,156,52,68,786,3674,2093,2093,562,1509


In [9]:
business_data.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,oaboaRBUgGjbo2kfUIKDLQ,Mike's Ice Cream,129 2nd Ave N,Nashville,TN,37201,36.162649,-86.775973,4.5,593,1,"{'AgesAllowed': None, 'Alcohol': 'u'none'', 'A...","Ice Cream & Frozen Yogurt, Coffee & Tea, Resta...","{'Friday': '8:0-0:0', 'Monday': '8:0-23:0', 'S..."
1,sB45WFgysT617bKWP_WJwA,Budweiser Brewery Experience,1200 Lynch St,Saint Louis,MO,63118,38.600197,-90.213538,4.5,605,1,"{'AgesAllowed': None, 'Alcohol': None, 'Ambien...","Bars, Beer Gardens, Food, Breweries, Nightlife...","{'Friday': '9:0-17:0', 'Monday': '0:0-0:0', 'S..."
2,SZU9c8V2GuREDN5KgyHFJw,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,"{'AgesAllowed': None, 'Alcohol': ''beer_and_wi...","Live/Raw Food, Restaurants, Seafood, Beer Bar,...","{'Friday': '11:0-21:0', 'Monday': '0:0-0:0', '..."
3,ORL4JE6tz3rJxVqkdKfegA,Gaylord Opryland Resort & Convention Center,2800 Opryland Dr,Nashville,TN,37214,36.211592,-86.694319,3.0,1639,1,"{'AgesAllowed': None, 'Alcohol': ''full_bar'',...","Venues & Event Spaces, Performing Arts, Arts &...","{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa..."
4,dChRGpit9fM_kZK5pafNyA,The Love,130 S 18th St,Philadelphia,PA,19103,39.950656,-75.170899,4.0,618,1,"{'AgesAllowed': None, 'Alcohol': 'u'full_bar''...","Restaurants, American (New), Breakfast & Brunc...","{'Friday': '16:0-21:0', 'Monday': '0:0-0:0', '..."


In [10]:
#Merge de reviews y business
#Asociamos las reseñas con los negocios por business_id
business_reviews = pd.merge(
    business_data[['business_id', 'name', 'categories', 'city']],
    reviews_data[['business_id', 'text']],
    on='business_id',
    how='inner'
)

In [11]:
business_reviews_grouped = business_reviews.groupby('business_id')['text'].apply(lambda x: ' '.join(x)).reset_index()

business_reviews_grouped = pd.merge(
    business_reviews_grouped,
    business_data[['business_id', 'name', 'categories', 'city']],
    on='business_id',
    how='inner'
)

In [12]:
#Cálculo de la matriz TF-IDF
#Vectorizamos las reseñas agrupadas
tfidf = TfidfVectorizer(stop_words='english')
reviews_matrix = tfidf.fit_transform(business_reviews_grouped['text'])

In [13]:
#Cantidad de veces que se repite cada término
tfidf.vocabulary_

{'finally': 16060,
 'chance': 7960,
 'dine': 12492,
 'indeblue': 21392,
 've': 43973,
 'sampled': 35756,
 'drums': 13486,
 'heaven': 19763,
 'samosas': 35753,
 'events': 14975,
 'highly': 20049,
 'anticipating': 2835,
 'experience': 15228,
 'let': 24091,
 'just': 22801,
 'say': 36023,
 'disappointed': 12589,
 'evening': 14965,
 'started': 39339,
 'bar': 4148,
 'happy': 19450,
 'hour': 20590,
 'group': 18889,
 'ordered': 29164,
 'appetizers': 2969,
 'share': 36928,
 'including': 21328,
 'spinach': 38901,
 'chaat': 7888,
 'barbeque': 4170,
 'pork': 31842,
 'spare': 38713,
 'rib': 34790,
 'samosa': 35752,
 'rolls': 35172,
 'vegetable': 44002,
 'naan': 27646,
 'pizza': 31321,
 'appetizer': 2968,
 'drinks': 13417,
 'menu': 26189,
 'comfort': 9402,
 'zone': 46425,
 'glass': 18160,
 'pinot': 31205,
 'grigio': 18788,
 'definitely': 11763,
 'new': 28018,
 'favorite': 15716,
 'philly': 30916,
 'reasonable': 33637,
 'absolutely': 1456,
 'delicious': 11861,
 'knew': 23282,
 'loved': 24795,
 'stand

In [14]:
doc = 0
feature_index = reviews_matrix[doc,:].nonzero()[1]
tfidf_scores = zip(feature_index, [reviews_matrix[doc, x] for x in feature_index])

In [15]:
feature_names = tfidf.get_feature_names_out()
tfidf_scores_sorted=[]
for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
    tfidf_scores_sorted.append([w,s])
    print(w,s)
tfidf_scores_sorted

finally 0.00935095828912767
chance 0.012067812629244825
dine 0.024615297382434388
indeblue 0.5822499760337175
ve 0.03531167030870835
sampled 0.01596888156640879
drums 0.13631579589709217
heaven 0.07175349226146412
samosas 0.15824388662649952
events 0.01663268127355413
highly 0.009737468555478013
anticipating 0.02775137153958259
experience 0.03303407644266412
let 0.008360841054347123
just 0.021645756245648105
say 0.006862653807362942
disappointed 0.009976224638419441
evening 0.020576929849809647
started 0.025298565849687708
bar 0.04060317005373618
happy 0.08296627869994504
hour 0.09308138599777689
group 0.025650617594228056
ordered 0.017937506445347497
appetizers 0.023373454901941336
share 0.02003093979484779
including 0.01008487413480629
spinach 0.09417628624567692
chaat 0.17616360598553601
barbeque 0.0255544968180904
pork 0.03913738368582647
spare 0.021061103562573118
rib 0.015297956547313498
samosa 0.06815789794854608
rolls 0.04382258220236402
vegetable 0.03264677493586331
naan 0.252

[['finally', 0.00935095828912767],
 ['chance', 0.012067812629244825],
 ['dine', 0.024615297382434388],
 ['indeblue', 0.5822499760337175],
 ['ve', 0.03531167030870835],
 ['sampled', 0.01596888156640879],
 ['drums', 0.13631579589709217],
 ['heaven', 0.07175349226146412],
 ['samosas', 0.15824388662649952],
 ['events', 0.01663268127355413],
 ['highly', 0.009737468555478013],
 ['anticipating', 0.02775137153958259],
 ['experience', 0.03303407644266412],
 ['let', 0.008360841054347123],
 ['just', 0.021645756245648105],
 ['say', 0.006862653807362942],
 ['disappointed', 0.009976224638419441],
 ['evening', 0.020576929849809647],
 ['started', 0.025298565849687708],
 ['bar', 0.04060317005373618],
 ['happy', 0.08296627869994504],
 ['hour', 0.09308138599777689],
 ['group', 0.025650617594228056],
 ['ordered', 0.017937506445347497],
 ['appetizers', 0.023373454901941336],
 ['share', 0.02003093979484779],
 ['including', 0.01008487413480629],
 ['spinach', 0.09417628624567692],
 ['chaat', 0.176163605985536

In [16]:
tfidf_scores_sorted_df = pd.DataFrame(tfidf_scores_sorted)
tfidf_scores_sorted_df.columns = ['token', 'val']
tfidf_scores_sorted_df.sort_values(by = 'val', ascending = False, inplace = True)
tfidf_scores_sorted_df.head(20)

,token,val
3,indeblue,0.582250
36,naan,0.252584
64,indian,0.214959
28,chaat,0.176164
8,samosas,0.158244
98,lamb,0.139952
6,drums,0.136316
362,tikka,0.115021
518,dosa,0.105698
37,pizza,0.101919


In [17]:
#Cálculo de la matriz de similitud coseno
similarity_matrix = cosine_similarity(reviews_matrix, reviews_matrix)
similarity_matrix

array([[1.        , 0.11514651, 0.09381649, ..., 0.04879991, 0.07836999,
        0.02802166],
       [0.11514651, 1.        , 0.07915939, ..., 0.0650769 , 0.07849669,
        0.02552422],
       [0.09381649, 0.07915939, 1.        , ..., 0.04018243, 0.08460318,
        0.04494629],
       ...,
       [0.04879991, 0.0650769 , 0.04018243, ..., 1.        , 0.05133013,
        0.02622616],
       [0.07836999, 0.07849669, 0.08460318, ..., 0.05133013, 1.        ,
        0.0317808 ],
       [0.02802166, 0.02552422, 0.04494629, ..., 0.02622616, 0.0317808 ,
        1.        ]])

In [18]:
#Función de recomendación
def recommend_restaurants_based_on_reviews(restaurant_id, n=15):
    #Verifica si el restaurante existe
    if restaurant_id not in business_reviews_grouped['business_id'].values:
        raise ValueError(f"El restaurante con ID '{restaurant_id}' no se encuentra en los datos.")
    #Identifica el índice del restaurante ingresado
    restaurant_index = business_reviews_grouped[business_reviews_grouped['business_id'] == restaurant_id].index[0]
    #Calcula la similitud del restaurante ingresado con todos los demás
    similarity_score = list(enumerate(similarity_matrix[restaurant_index]))
    #Ordena los restaurantes por puntaje de similitud en orden descendente
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    #Obtiene los índices de los `n` restaurantes más similares, ignorando el primero
    similarity_score = similarity_score[1:n+1]
    restaurant_indices = [i[0] for i in similarity_score]
    #Obtiene información de los restaurantes similares
    return business_reviews_grouped.iloc[restaurant_indices][['business_id', 'name', 'categories', 'city']]

In [19]:
#Prueba de la función de recomendación
restaurant_id = "ORL4JE6tz3rJxVqkdKfegA"
recommended_restaurants = recommend_restaurants_based_on_reviews(restaurant_id, n=15)
recommended_restaurants.loc[:,:]

,business_id,name,categories,city
1286,wz8ZPfySQczcPgSyd33-HQ,Grand Sierra Resort and Casino,"Nightlife, Hotels & Travel, Arts & Entertainme...",Reno
1047,lZX5_8qspEXtMJGFQ0-Mow,Philadelphia Marriott Downtown,"Event Planning & Services, Hotels & Travel, Ve...",Philadelphia
945,hRskO0RDRW3Cq3k7_Kggcg,Le Pavillon Hotel,"Hotels, Hotels & Travel, Restaurants, Venues &...",New Orleans
1035,l-rGtJt0E7PAklT0IK7oFQ,Four Seasons Resort The Biltmore Santa Barbara,"Hotels & Travel, Hotels, Event Planning & Serv...",Santa Barbara
1333,zljfjg_zTL3ceGi_Vr4OFw,Sheraton New Orleans Hotel,"Cajun/Creole, Hotels, Venues & Event Spaces, H...",New Orleans
38,0M1l6p96IffjQi9EZsdVhw,New Orleans Marriott,"American (Traditional), Event Planning & Servi...",New Orleans
73,1aj1hnqPt3rqoB8tVRHLEA,Hotel Monteleone,"Hotels & Travel, Hotels, Event Planning & Serv...",New Orleans
318,D5V0Fawd6ODVgqCY8xngsw,Loews Philadelphia Hotel,"Event Planning & Services, Hotels & Travel, Re...",Philadelphia
11,-OKB11ypR4C8wWlonBFIGw,Atlantis Casino Resort Spa,"Casinos, Hotels & Travel, Arts & Entertainment...",Reno
573,PY9GRfzr4nTZeINf346QOw,Peppermill Reno,"Event Planning & Services, Casinos, Beauty & S...",Reno


In [20]:
recs_df= recommend_restaurants_based_on_reviews(restaurant_id, n=15)
recs_df.loc[~recs_df.city.str.contains('Philadelphia'),:]

,business_id,name,categories,city
1286,wz8ZPfySQczcPgSyd33-HQ,Grand Sierra Resort and Casino,"Nightlife, Hotels & Travel, Arts & Entertainme...",Reno
945,hRskO0RDRW3Cq3k7_Kggcg,Le Pavillon Hotel,"Hotels, Hotels & Travel, Restaurants, Venues &...",New Orleans
1035,l-rGtJt0E7PAklT0IK7oFQ,Four Seasons Resort The Biltmore Santa Barbara,"Hotels & Travel, Hotels, Event Planning & Serv...",Santa Barbara
1333,zljfjg_zTL3ceGi_Vr4OFw,Sheraton New Orleans Hotel,"Cajun/Creole, Hotels, Venues & Event Spaces, H...",New Orleans
38,0M1l6p96IffjQi9EZsdVhw,New Orleans Marriott,"American (Traditional), Event Planning & Servi...",New Orleans
73,1aj1hnqPt3rqoB8tVRHLEA,Hotel Monteleone,"Hotels & Travel, Hotels, Event Planning & Serv...",New Orleans
11,-OKB11ypR4C8wWlonBFIGw,Atlantis Casino Resort Spa,"Casinos, Hotels & Travel, Arts & Entertainment...",Reno
573,PY9GRfzr4nTZeINf346QOw,Peppermill Reno,"Event Planning & Services, Casinos, Beauty & S...",Reno
1237,v1Uesklh8DpEufYOhTq4iA,Mar Monte Hotel - Hyatt,"Hotels, Restaurants, Beauty & Spas, Hotels & T...",Santa Barbara
1216,u-wkhjV7CfvGJVY7hGULjw,Hilton New Orleans Riverside,"Event Planning & Services, Hotels & Travel, Ho...",New Orleans


In [21]:
def calculate_mse(user_id, reviews_data, similarity_matrix, user_ids):
    #Filtra las calificaciones del usuario
    user_ratings = reviews_data[reviews_data['user_id'] == user_id]
    #Crea un diccionario para almacenar predicciones
    predictions = {}
    #Obtiene el índice del usuario en la lista de user_ids (usando np.where)
    try:
        user_idx = np.where(user_ids == user_id)[0][0]
    except IndexError:
        print(f"Error: El usuario {user_id} no se encuentra en user_ids")
        return None
    for business_id in user_ratings['business_id']:
        #Busca las calificaciones de otros usuarios para el mismo negocio
        other_users_ratings = reviews_data[reviews_data['business_id'] == business_id]
        #Calcula la predicción basada en similitud de coseno
        weighted_sum = 0
        similarity_sum = 0
        for other_user in other_users_ratings['user_id']:
            if other_user != user_id:
                try:
                    #Obtiene el índice del otro usuario en la lista de user_ids
                    other_user_idx = np.where(user_ids == other_user)[0][0]
                    #Verifica que el índice esté dentro de los límites
                    if other_user_idx < similarity_matrix.shape[1]:  #Verifica si el índice está dentro de los límites
                        sim = similarity_matrix[user_idx, other_user_idx]
                    else:
                        sim = 0  #Si el índice está fuera de los límites, asignamos similitud 0
                except IndexError:
                    sim = 0  #Si ocurre un error en la obtención del índice, asignamos similitud 0
                if sim > 0:  #Solo considera usuarios similares
                    weighted_sum += sim * other_users_ratings[other_users_ratings['user_id'] == other_user]['stars'].values[0]
                    similarity_sum += sim
        #Si hay usuarios similares, calcula la predicción
        if similarity_sum > 0:
            prediction = weighted_sum / similarity_sum
            predictions[business_id] = prediction
    #Calcula el MSE solo para los negocios que el usuario calificó
    mse_sum = 0
    n = 0  #Contador de negocios evaluados
    for business_id, real_rating in zip(user_ratings['business_id'], user_ratings['stars']):
        if business_id in predictions:
            predicted_rating = predictions[business_id]
            mse_sum += (real_rating - predicted_rating) ** 2
            n += 1
    return mse_sum / n if n > 0 else None

In [24]:
import numpy as np
user_ids = reviews_data['user_id'].unique()  # Obtener la lista de usuarios
user_id = 'Bf87HcPERF9yiSjb2tQBqw'  # ID del usuario
mse_value = calculate_mse(user_id, reviews_data, similarity_matrix, user_ids)
print(f'MSE para el usuario {user_id}: {mse_value}')

MSE para el usuario Bf87HcPERF9yiSjb2tQBqw: 0.7912646400444805


In [25]:
#Función para guardar el modelo (matriz de similitud)
def save_model(model, filename):
    with open(filename, 'wb') as f:
        pickle.dump(model, f)
    print(f"Modelo guardado en {filename}")
#Guarda la similarity_matrix en un archivo pickle
save_model(similarity_matrix, 'similarity_matrix.pkl')

Modelo guardado en similarity_matrix.pkl


In [26]:
#Función para cargar el modelo desde pickle
def load_model(filename):
    with open(filename, 'rb') as f:
        model = pickle.load(f)
    print(f"Modelo cargado desde {filename}")
    return model
#Carga la similarity_matrix desde el archivo pickle
loaded_similarity_matrix = load_model('similarity_matrix.pkl')

Modelo cargado desde similarity_matrix.pkl
